# Tabnet Image Model

In [1]:
import tabnet
from load_data import *
import os
from tqdm import tqdm_notebook
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
tf.compat.v1.enable_eager_execution()


# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

Init Plugin
Init Graph Optimizer
Init Kernel


We only consider a model with the best hyperparameters identified for the individual models

In [2]:
# Load data for best fold of Tabnet
X_train, X_test, X_val, y_train, y_test, y_val = load_data_fold(fold=4, drop_id = False)

## Input process for Image Model

In [3]:
# load paths and labels
img_folder = "data/images_resized"
img_df = pd.read_csv("data/img_paths.csv")
img_df.reset_index(drop=True, inplace = True)

# load label book
label_cat = ["bathroom", "bedroom", "dining", "hallway", "kitchen", "living"]
label = np.arange(6)
label_book = pd.DataFrame({"label": label_cat, "categorical_label": label})

In [4]:
# prices
url_listing = "http://data.insideairbnb.com/ireland/leinster/dublin/2021-11-07/data/listings.csv.gz"
listings = pd.read_csv(url_listing)
urls = listings["listing_url"]
ids = listings["id"]
price = listings["price"]
price = price.str.replace("$","")
price = price.str.replace(",","")
price = price.astype(float)
listings["price"] = price
listings["log_price"] = np.log(price)
listings = listings[listings["price"]<500]


In [5]:
price_df = listings[["log_price", "id"]]
df = pd.merge(img_df, price_df, on = "id", how = "left")
df.head()

,img_path,id,img_no,label,log_price
0,44077_0.png,44077,0,4.0,4.174387
1,44077_1.png,44077,1,5.0,4.174387
2,44077_2.png,44077,2,1.0,4.174387
3,44077_3.png,44077,3,1.0,4.174387
4,44077_4.png,44077,4,1.0,4.174387


In [6]:
filter = np.any(df.isna(), axis = 1)
df = df[~filter]

# drop "others"
filter = df["label"] == 6.0
df = df[~filter]

In [7]:
bool = []
for id in df["id"]:
    tmp = df[df["id"] == id]
    if len(np.unique(tmp["label"])) >= 4:
        bool.append(True)
    else:
        bool.append(False)

In [8]:
df_new = df[bool]
np.unique(df_new["id"].values).shape
df = df_new

In [9]:
df.shape

(56704, 5)

In [10]:
def input_pipeline(room = 0, df = df):
    ids = []
    features = []
    
    # FILTER DF
    df_room = df[df["label"] == room]
    
    # RESNET
    resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg", input_shape = (None,None,3))
    resnet_pre = keras.applications.resnet50.preprocess_input
    resnet.trainable = False
    
    for id in tqdm(np.unique(df["id"])):
        filter = df_room["id"] == id  

        try:
            l = []
            if filter.sum() == 0:
                dummy_image = np.zeros((1,256,256,3))
                dummy_image = resnet_pre(dummy_image)
                dummy_image = resnet(dummy_image)
                l.append(dummy_image)                
            else:
                path_id = df_room["img_path"][filter]
                for p in path_id:
                    img_tmp = plt.imread("data/images_resized/"+p)
                    img_tmp = np.expand_dims(img_tmp, axis = 0)
                    img_tmp = resnet_pre(img_tmp)
                    img_tmp = resnet(img_tmp)
                    l.append(img_tmp)
            l = np.stack(l)
            l = np.max(l, axis = 0)
            features.append(l)
            ids.append(id)
        except:
            dummy_image = np.zeros((1,256,256,3))
            dummy_image = resnet_pre(dummy_image)
            dummy_image = resnet(dummy_image)
            l.append(dummy_image)
            l = np.stack(l)
            l = np.max(l, axis = 0)
            features.append(l)
            ids.append(id)
            continue
    features = np.squeeze(np.stack(features))
    filter = np.nonzero(features.sum(axis = 0))[0]
    features = features[:,filter]
    print(len(filter), " features are nonzero.")
    features = features.tolist()
     
    return features, ids

In [11]:
def data_generator(df):
    basis_df = df[["id","log_price"]]
    basis_df = basis_df.drop_duplicates()
    
    features = []
    ids = []
    
    for i in tqdm(np.unique(df["label"])):
        feat_cat, ids_cat = input_pipeline(i,df)
        df_tmp = pd.DataFrame({"features_"+str(i): feat_cat, "id": ids_cat})
        basis_df = pd.merge(basis_df, df_tmp, on = "id", how = "left")
    return basis_df
    

In [12]:
def data_generator(df):
    counter_overall_dummy = 0
    counter_overall_img = 0

    def input_pipeline(room = 0, df = df):
        ids = []
        features = []
        
        # FILTER DF
        df_room = df[df["label"] == room]
        
        # RESNET
        resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg", input_shape = (None,None,3))
        resnet_pre = keras.applications.resnet50.preprocess_input
        resnet.trainable = False
        counter_dummy = 0
        counter_img = 0
        for id in tqdm_notebook(np.unique(df["id"])):
            filter = df_room["id"] == id  

            try:
                l = []
                if filter.sum() == 0:
                    dummy_image = np.zeros((1,256,256,3))
                    dummy_image = resnet_pre(dummy_image)
                    dummy_image = resnet(dummy_image)
                    l.append(dummy_image)
                    counter_dummy += 1                
                else:
                    path_id = df_room["img_path"][filter]
                    for p in path_id:
                        img_tmp = plt.imread("data/images_resized/"+p)
                        img_tmp = np.expand_dims(img_tmp, axis = 0)
                        img_tmp = resnet_pre(img_tmp)
                        img_tmp = resnet(img_tmp)
                        l.append(img_tmp)
                        counter_img += 1
                l = np.stack(l)
                l = np.max(l, axis = 0)
                features.append(l)
                ids.append(id)
            except:
                dummy_image = np.zeros((1,256,256,3))
                dummy_image = resnet_pre(dummy_image)
                dummy_image = resnet(dummy_image)
                l.append(dummy_image)
                l = np.stack(l)
                l = np.max(l, axis = 0)
                features.append(l)
                ids.append(id)
                continue
        features = np.squeeze(np.stack(features))
        filter = np.nonzero(features.sum(axis = 0))[0]
        features = features[:,filter]
        print(len(filter), " features are nonzero.")
        features = features.tolist()
        print(counter_dummy, "dummy images were added.")
        return features, ids, counter_dummy, counter_img

    basis_df = df[["id","log_price"]]
    basis_df = basis_df.drop_duplicates()
    
    features = []
    ids = []
    
    for i in tqdm_notebook(np.unique(df["label"])):
        feat_cat, ids_cat, counter_dummy, counter_img = input_pipeline(i,df)
        df_tmp = pd.DataFrame({"features_"+str(i): feat_cat, "id": ids_cat})
        basis_df = pd.merge(basis_df, df_tmp, on = "id", how = "left")
        counter_overall_dummy += counter_dummy
        counter_overall_img += counter_img
    print(counter_overall_dummy)
    print(counter_overall_img)
    return basis_df
    

In [13]:
final_df = data_generator(df)

/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_3389/1717659073.py:66: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(np.unique(df["label"])):


  0%|          | 0/6 [00:00<?, ?it/s]

2022-02-22 15:23:59.798978: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-22 15:23:59.799395: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_3389/1717659073.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm_notebook(np.unique(df["id"])):


  0%|          | 0/4356 [00:00<?, ?it/s]

1416  features are nonzero.
266 dummy images were added.


/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_3389/1717659073.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm_notebook(np.unique(df["id"])):


  0%|          | 0/4356 [00:00<?, ?it/s]

In [ ]:
final_df.columns = ["id", "log_price", "bath", "bed", "dining", "hall", "kitchen", "living"]

## Input process for TabNet

In [ ]:

# set columns for tabnet
bin_col = [col for col in X_train if np.isin(X_train[col].unique(), [0, 1]).all()]
num_col = [col for col in X_train if ~np.isin(X_train[col].unique(), [0, 1]).all()]
col_names = bin_col + num_col
feature_columns = []
col_names.remove("id")
for col in col_names:
    feature_columns.append(tf.feature_column.numeric_column(col))

In [ ]:
# merge data for tabnet and images to get consistent ids
data_df_train = pd.merge(final_df, X_train, on="id", how="inner")
data_df_train.dropna(inplace = True)
print("TRAIN SHAPE: ", data_df_train.shape)

data_df_test = pd.merge(final_df, X_test, on="id", how="inner")
data_df_test.dropna(inplace = True)
print("TEST SHAPE: ", data_df_test.shape)

data_df_val = pd.merge(final_df, X_val, on="id", how="inner")
data_df_val.dropna(inplace = True)
print("VAL SHAPE: ", data_df_val.shape)

# extract consistent columns
X_train_tab = data_df_train.filter(col_names)
X_test_tab = data_df_test.filter(col_names)
X_val_tab = data_df_val.filter(col_names)

data_df_train.drop(col_names, axis = 1, inplace = True)
data_df_test.drop(col_names, axis = 1, inplace = True)
data_df_val.drop(col_names, axis = 1, inplace = True)

# extract consistent price
y_train = data_df_train.pop("log_price")
y_test = data_df_test.pop("log_price")
y_val = data_df_val.pop("log_price")

# drop id
X_train = data_df_train.drop("id", axis = 1)
X_test = data_df_test.drop("id", axis = 1)
X_val = data_df_val.drop("id", axis = 1)


TRAIN SHAPE:  (2710, 77)
TEST SHAPE:  (853, 77)
VAL SHAPE:  (706, 77)


In [ ]:
def transform(ds):
    # tabnet
    features = tf.unstack(ds["features"])
    features = dict(zip(col_names, features))

    # images
    bath = tf.unstack(ds["bath"])
    bed = tf.unstack(ds["bed"])
    dining = tf.unstack(ds["dining"])
    hall = tf.unstack(ds["hall"])
    kitchen = tf.unstack(ds["kitchen"])
    living = tf.unstack(ds["living"])
    
    prices = ds["price"]
    
    y = prices
    return (bath, bed, dining, hall, kitchen, living, features), y

Construct tensorflow dataset

In [ ]:
train_size = int(X_train.shape[0] * 0.9)
batch_size = int(X_train.shape[0] * 0.1)

# X_train, X_test, y_train, y_test = train_test_split(features, prices, random_state = 123, test_size = 0.2)
# X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.2, random_state=1) # 0.25 x 0.8 = 0.2

data_train = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_train["bath"])),
                                                 "bed": np.squeeze(np.stack(X_train["bed"])),
                                                 "dining": np.squeeze(np.stack(X_train["dining"])),
                                                 "hall": np.squeeze(np.stack(X_train["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_train["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_train["living"])),
                                                 "features": X_train_tab,
                                                 "price": y_train})
data_train = data_train.cache()
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(y_train))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(batch_size)

data_test = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_test["bath"])),
                                                 "bed": np.squeeze(np.stack(X_test["bed"])),
                                                 "dining": np.squeeze(np.stack(X_test["dining"])),
                                                 "hall": np.squeeze(np.stack(X_test["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_test["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_test["living"])),
                                                 "features": X_test_tab,
                                                 "price": y_test})
data_test = data_test.cache()
test_dataset = data_test.take(len(y_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(batch_size)

data_val = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_val["bath"])),
                                                 "bed": np.squeeze(np.stack(X_val["bed"])),
                                                 "dining": np.squeeze(np.stack(X_val["dining"])),
                                                 "hall": np.squeeze(np.stack(X_val["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_val["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_val["living"])),
                                                 "features": X_val_tab,
                                                 "price": y_val})
data_val = data_val.cache()
val_dataset = data_val.take(len(y_val))
val_dataset = val_dataset.map(transform)
val_dataset = val_dataset.batch(batch_size)

In [ ]:
from tensorflow.keras import backend
class weight_constr(tf.keras.constraints.Constraint):
  """Constrains weight tensors to be centered around `ref_value`."""

  def __init__(self):
    self.ref_value = 1

  def __call__(self, w):
    nonneg = w * tf.cast(tf.greater_equal(w, 0.), backend.floatx())
    sum_w = tf.reduce_sum(nonneg)
    nonneg_one = nonneg/sum_w
    return nonneg_one


Model Setup

In [ ]:
# setup model
class Compound_model(tf.keras.Model):

  def __init__(self,feature_columns, dropout = 0, l2 = 0, nodes1 = 512, nodes2 = 1,
                 num_features=None,
                 feature_dim=32,
                 output_dim=32,
                 num_decision_steps=3,
                 relaxation_factor=1.5,
                 sparsity_coefficient=1e-5,
                 norm_type='group',
                 batch_momentum=0.98,
                 virtual_batch_size=None,
                 num_groups=1,
                 epsilon=1e-5):
    super().__init__()
    # TabNet
    self.tabnet = tabnet.TabNet(feature_columns=feature_columns,
                          num_features=num_features,
                          feature_dim=feature_dim,
                          output_dim=output_dim,
                          num_decision_steps=num_decision_steps,
                          relaxation_factor=relaxation_factor,
                          sparsity_coefficient=sparsity_coefficient,
                          norm_type=norm_type,
                          batch_momentum=batch_momentum,
                          virtual_batch_size=virtual_batch_size,
                          num_groups=num_groups,
                          epsilon=epsilon)
   
    
    # bathroom
    self.bn1_bath = tf.keras.layers.BatchNormalization()
    self.drop1_bath = tf.keras.layers.Dropout(dropout)
    self.dense1_bath = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bath = tf.keras.layers.BatchNormalization()
    self.drop2_bath = tf.keras.layers.Dropout(dropout)
    #self.dense2_bath = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bath = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
 
    # bedroom
    self.bn1_bed = tf.keras.layers.BatchNormalization()
    self.drop1_bed = tf.keras.layers.Dropout(dropout)
    self.dense1_bed = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bed = tf.keras.layers.BatchNormalization()
    self.drop2_bed = tf.keras.layers.Dropout(dropout)
    #self.dense2_bed = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bed = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
       
      # dining
    self.bn1_dining = tf.keras.layers.BatchNormalization()
    self.drop1_dining = tf.keras.layers.Dropout(dropout)
    self.dense1_dining = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_dining = tf.keras.layers.BatchNormalization()
    self.drop2_dining = tf.keras.layers.Dropout(dropout)
    #self.dense2_dining = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_dining = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
   
    # hall
    self.bn1_hall = tf.keras.layers.BatchNormalization()
    self.drop1_hall = tf.keras.layers.Dropout(dropout)
    self.dense1_hall = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_hall = tf.keras.layers.BatchNormalization()
    self.drop2_hall = tf.keras.layers.Dropout(dropout)
    #self.dense2_hall = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_hall = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
    
    # kitchen
    self.bn1_kitchen = tf.keras.layers.BatchNormalization()
    self.drop1_kitchen = tf.keras.layers.Dropout(dropout)
    self.dense1_kitchen = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_kitchen = tf.keras.layers.BatchNormalization()
    self.drop2_kitchen = tf.keras.layers.Dropout(dropout)
    #self.dense2_kitchen = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_kitchen = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
  
    # livingroom
    self.bn1_living = tf.keras.layers.BatchNormalization()
    self.drop1_living = tf.keras.layers.Dropout(dropout)
    self.dense1_living = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_living = tf.keras.layers.BatchNormalization()
    self.drop2_living = tf.keras.layers.Dropout(dropout)
    #self.dense2_living = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_living = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
    
    # bring all images together
    self.bn_img = tf.keras.layers.BatchNormalization()
    self.drop_img = tf.keras.layers.Dropout(dropout)
    self.dense_img = tf.keras.layers.Dense(1, activation = tf.nn.relu)
    
    # bring features of TabNet together
    self.dense_tabnet = tf.keras.layers.Dense(1, activation = tf.nn.relu)
        
    # final prediction
    self.bn_final = tf.keras.layers.BatchNormalization()
    self.drop_final = tf.keras.layers.Dropout(dropout)
    self.dense_final = tf.keras.layers.Dense(1, kernel_constraint =weight_constr())
    
  def call(self, inputs, training = None):
    
    # bathroom
    x = self.bn1_bath(inputs[0])
    x = self.drop1_bath(x)
    x = self.dense1_bath(x)
    x = self.bn2_bath(x)
    x = self.drop2_bath(x)
    bath_out = self.dense2_bath(x)
    
    
    # bedroom
    x = self.bn1_bed(inputs[1])
    x = self.drop1_bed(x)
    x = self.dense1_bed(x)
    x = self.bn2_bed(x)
    x = self.drop2_bed(x)
    bed_out = self.dense2_bed(x)
    
    # diningroom
    x = self.bn1_dining(inputs[2])
    x = self.drop1_dining(x)
    x = self.dense1_dining(x)
    x = self.bn2_dining(x)
    x = self.drop2_dining(x)
    dining_out = self.dense2_dining(x)
    
    # hallroom
    x = self.bn1_hall(inputs[3])
    x = self.drop1_hall(x)
    x = self.dense1_hall(x)
    x = self.bn2_hall(x)
    x = self.drop2_hall(x)
    hall_out = self.dense2_hall(x)
    
    # kitchen
    x = self.bn1_kitchen(inputs[4])
    x = self.drop1_kitchen(x)
    x = self.dense1_kitchen(x)
    x = self.bn2_kitchen(x)
    x = self.drop2_kitchen(x)
    kitchen_out = self.dense2_kitchen(x)
    
    # livingroom
    x = self.bn1_living(inputs[5])
    x = self.drop1_living(x)
    x = self.dense1_living(x)
    x = self.bn2_living(x)
    x = self.drop2_living(x)
    living_out = self.dense2_living(x)
    
    
    # tabnet
    self.activations = self.tabnet(inputs[6], training=True)
    out_tabnet = self.dense_tabnet(self.activations)

    # join images
    out_img = tf.keras.layers.concatenate([bath_out, bed_out, dining_out, hall_out, kitchen_out, living_out])
    out_img = self.bn_img(out_img)
    out_img = self.drop_img(out_img)
    out_img = self.dense_img(out_img)

    # join
    out = tf.keras.layers.concatenate([out_img, out_tabnet])
    out = self.bn_final(out)
    out = self.drop_final(out)
    return self.dense_final(out)

In [ ]:
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))  
  return r2

## Train Compound from Scratch

In [ ]:
tf.random.set_seed(2)
model = Compound_model(feature_columns = feature_columns,
                                output_dim=120, feature_dim=125, num_groups=1,
                                num_decision_steps=2, relaxation_factor=1.5,
                                sparsity_coefficient=1e-05, dropout= 0.2, nodes1 = 512, nodes2 = 16, l2 = 0)
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.95, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss='mse' , metrics=[R_squared,"mae"])

model.fit(train_dataset, validation_data = test_dataset, epochs=300, verbose=1)

[TabNet]: 5 features will be used for decision steps.
Epoch 1/300


NotImplementedError: in user code:

    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_2812/2176399485.py:152 call  *
        self.activations = self.tabnet(inputs[6], training=True)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tabnet/tabnet.py:224 call  *
        output_aggregated = tf.zeros([batch_size, self.output_dim])
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/util/dispatch.py:206 wrapper  **
        return target(*args, **kwargs)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/python3.9/site-packages/tensorflow/python/ops/array_ops.py:2911 wrapped
        tensor = fun(*args, **kwargs)
    /opt/homebrew/Caskroom/miniforge/base/envs/tensorflow_m1/lib/p

Evaluate test data

In [ ]:
model.evaluate(test_dataset)

4/4 [==============================] - 2s 228ms/step - loss: 9.0255 - R_squared: -24.2988 - mae: 2.4546


[9.025516510009766, -24.298797607421875, 2.4545772075653076]

Evaluate weights of final layer

In [ ]:
model.layers[-1].get_weights()

[array([[-0.       ],
        [ 1.0000001]], dtype=float32),
 array([0.19783714], dtype=float32)]

# Reload optimal weights from best TabNet and Image Model

In [ ]:
model = Compound_model(feature_columns = feature_columns,
                                output_dim=120, feature_dim=125, num_groups=1,
                                num_decision_steps=2, relaxation_factor=1.5,
                                sparsity_coefficient=1e-05, dropout= 0.2, nodes1 = 512, nodes2 = 1, l2 = 0)
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.95, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss='mse' , metrics=[R_squared,"mae"])

model.fit(train_dataset, validation_data = test_dataset, epochs=2, verbose=1)

## Load best TabNet

In [179]:
X_train_tn, X_test_tn, X_val_tn, y_train_tn, y_test_tn, y_val_tn = load_data(for_dendro = False)

def transform(ds):
    features = tf.unstack(ds["features"])
    prices = ds["price"]

    x = dict(zip(col_names, features))
    y = prices
    return x, y

def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))
  
  return r2

bin_col = [col for col in X_train_tn if np.isin(X_train_tn[col].unique(), [0, 1]).all()]
num_col = [col for col in X_train_tn if ~np.isin(X_train_tn[col].unique(), [0, 1]).all()]
col_names = bin_col + num_col

train_size = int(X_train_tn.shape[0] * 0.9)
batch_size = int(X_train_tn.shape[0] * 0.1)

data_train = tf.data.Dataset.from_tensor_slices({"features": X_train_tn, "price": y_train_tn})
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(X_train_tn))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(batch_size)

data_test = tf.data.Dataset.from_tensor_slices({"features": X_val_tn, "price": y_val_tn})
test_dataset = data_test.take(len(X_val_tn))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(batch_size)


feature_columns = []

for col in col_names:
    feature_columns.append(tf.feature_column.numeric_column(col))
od = 120
fd =  125
nds = 2
rf = 1.5
sc = 1e-05

lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.95, staircase=False)
#lr = 0.01
optimizer = tf.keras.optimizers.Adam(lr)

model_tn = tabnet.TabNetRegression(feature_columns, num_regressors=1,
                                output_dim=od, feature_dim=fd, num_groups=1,
                                num_decision_steps=nds, relaxation_factor=rf)

model_tn.compile(optimizer, loss='mse' , metrics=[R_squared, "mse", "mae"])
model_tn.fit(test_dataset)
model_tn.load_weights("TabNet_Selected/best_model3.hdf5")

[TabNet]: 5 features will be used for decision steps.


2022-02-21 21:27:00.126076: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 3s 138ms/step - loss: 12.5054 - R_squared: -24.5086 - mse: 12.5054 - mae: 3.1016


## Load best Image Model

In [182]:
def transform(ds):
    bath = tf.unstack(ds["bath"])
    bed = tf.unstack(ds["bed"])
    dining = tf.unstack(ds["dining"])
    hall = tf.unstack(ds["hall"])
    kitchen = tf.unstack(ds["kitchen"])
    living = tf.unstack(ds["living"])
   # others = tf.unstack(ds["others"])

    prices = ds["price"]
    
    y = prices
    return (bath, bed, dining, hall, kitchen, living), y

batch_size = 128

# Load data
X_train, X_test, X_val, y_train, y_test, y_val = load_data_fold(fold=4, drop_id = False)

train_ids = X_train["id"]
test_ids = X_test["id"]
val_ids = X_val["id"]

X_train = pd.merge(train_ids, final_df, on = "id", how = "left")
X_train = X_train.drop(["id","log_price"], axis = 1)
X_train.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
filter = np.any(X_train.isna(), axis = 1).values
X_train = X_train[~filter]
y_train = y_train[~filter]

X_test = pd.merge(test_ids, final_df, on = "id", how = "left")
X_test = X_test.drop(["id","log_price"], axis = 1)
X_test.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
filter = np.any(X_test.isna(), axis = 1).values
X_test = X_test[~filter]
y_test = y_test[~filter]

X_val = pd.merge(val_ids, final_df, on = "id", how = "left")
X_val = X_val.drop(["id","log_price"], axis = 1)
X_val.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
filter = np.any(X_val.isna(), axis = 1).values
X_val = X_val[~filter]
y_val = y_val[~filter]

data_train = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_train["bath"])),
                                                 "bed": np.squeeze(np.stack(X_train["bed"])),
                                                 "dining": np.squeeze(np.stack(X_train["dining"])),
                                                 "hall": np.squeeze(np.stack(X_train["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_train["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_train["living"])),
                                                 #"others": np.squeeze(np.stack(X_train["others"])),
                                                 "price": y_train})
data_train = data_train.cache()
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(y_train))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(batch_size)

data_test = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_test["bath"])),
                                                 "bed": np.squeeze(np.stack(X_test["bed"])),
                                                 "dining": np.squeeze(np.stack(X_test["dining"])),
                                                 "hall": np.squeeze(np.stack(X_test["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_test["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_test["living"])),
                                                # "others": np.squeeze(np.stack(X_test["others"])),
                                                 "price": y_test})
data_test = data_test.cache()
test_dataset = data_test.take(len(y_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(batch_size)

data_val = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_val["bath"])),
                                                 "bed": np.squeeze(np.stack(X_val["bed"])),
                                                 "dining": np.squeeze(np.stack(X_val["dining"])),
                                                 "hall": np.squeeze(np.stack(X_val["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_val["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_val["living"])),
                                                 #"others": np.squeeze(np.stack(X_val["others"])),
                                                 "price": y_val})
data_val = data_val.cache()
val_dataset = data_val.take(len(y_val))
val_dataset = val_dataset.map(transform)
val_dataset = val_dataset.batch(batch_size)

In [183]:
# setup model
class Img_model(tf.keras.Model):

  def __init__(self, dropout = 0, l2 = 0, nodes1 = 512, nodes2 = 1):
    super().__init__()
    
    # bathroom
    self.bn1_bath = tf.keras.layers.BatchNormalization()
    self.drop1_bath = tf.keras.layers.Dropout(dropout)
    self.dense1_bath = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bath = tf.keras.layers.BatchNormalization()
    self.drop2_bath = tf.keras.layers.Dropout(dropout)
    #self.dense2_bath = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bath = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
 
    # bedroom
    self.bn1_bed = tf.keras.layers.BatchNormalization()
    self.drop1_bed = tf.keras.layers.Dropout(dropout)
    self.dense1_bed = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bed = tf.keras.layers.BatchNormalization()
    self.drop2_bed = tf.keras.layers.Dropout(dropout)
    #self.dense2_bed = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bed = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
       
      # dining
    self.bn1_dining = tf.keras.layers.BatchNormalization()
    self.drop1_dining = tf.keras.layers.Dropout(dropout)
    self.dense1_dining = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_dining = tf.keras.layers.BatchNormalization()
    self.drop2_dining = tf.keras.layers.Dropout(dropout)
    #self.dense2_dining = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_dining = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
   
    # hall
    self.bn1_hall = tf.keras.layers.BatchNormalization()
    self.drop1_hall = tf.keras.layers.Dropout(dropout)
    self.dense1_hall = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_hall = tf.keras.layers.BatchNormalization()
    self.drop2_hall = tf.keras.layers.Dropout(dropout)
    #self.dense2_hall = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_hall = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
    
      # kitchen
    self.bn1_kitchen = tf.keras.layers.BatchNormalization()
    self.drop1_kitchen = tf.keras.layers.Dropout(dropout)
    self.dense1_kitchen = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_kitchen = tf.keras.layers.BatchNormalization()
    self.drop2_kitchen = tf.keras.layers.Dropout(dropout)
    #self.dense2_kitchen = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_kitchen = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
  
      # livingroom
    self.bn1_living = tf.keras.layers.BatchNormalization()
    self.drop1_living = tf.keras.layers.Dropout(dropout)
    self.dense1_living = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_living = tf.keras.layers.BatchNormalization()
    self.drop2_living = tf.keras.layers.Dropout(dropout)
    #self.dense2_living = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_living = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))

    # final prediction
    self.bn_final = tf.keras.layers.BatchNormalization()
    self.drop_final = tf.keras.layers.Dropout(dropout)
    self.dense_final = tf.keras.layers.Dense(1, kernel_constraint =weight_constr())
    
  def call(self, inputs, training = None):
    
    # bathroom
    x = self.bn1_bath(inputs[0])
    x = self.drop1_bath(x)
    x = self.dense1_bath(x)
    x = self.bn2_bath(x)
    x = self.drop2_bath(x)
    bath_out = self.dense2_bath(x)
    
    
    # bedroom
    x = self.bn1_bed(inputs[1])
    x = self.drop1_bed(x)
    x = self.dense1_bed(x)
    x = self.bn2_bed(x)
    x = self.drop2_bed(x)
    bed_out = self.dense2_bed(x)
    
    # diningroom
    x = self.bn1_dining(inputs[2])
    x = self.drop1_dining(x)
    x = self.dense1_dining(x)
    x = self.bn2_dining(x)
    x = self.drop2_dining(x)
    dining_out = self.dense2_dining(x)
    
    # hallroom
    x = self.bn1_hall(inputs[3])
    x = self.drop1_hall(x)
    x = self.dense1_hall(x)
    x = self.bn2_hall(x)
    x = self.drop2_hall(x)
    hall_out = self.dense2_hall(x)
    
    # kitchen
    x = self.bn1_kitchen(inputs[4])
    x = self.drop1_kitchen(x)
    x = self.dense1_kitchen(x)
    x = self.bn2_kitchen(x)
    x = self.drop2_kitchen(x)
    kitchen_out = self.dense2_kitchen(x)
    
    # livingroom
    x = self.bn1_living(inputs[5])
    x = self.drop1_living(x)
    x = self.dense1_living(x)
    x = self.bn2_living(x)
    x = self.drop2_living(x)
    living_out = self.dense2_living(x)
    
    # others
    # x = self.bn1_others(inputs[6])
    # x = self.drop1_others(x)
    # x = self.dense1_others(x)
    # x = self.bn2_others(x)
    # x = self.drop2_others(x)
    # others_out = self.dense2_others(x)

    # join
    out = tf.keras.layers.concatenate([bath_out, bed_out, dining_out, hall_out, kitchen_out, living_out])#, others_out])
    out = self.bn_final(out)
    out = self.drop_final(out)
    return self.dense_final(out)

In [184]:
model_img = Img_model(dropout = 0.2, l2 = 0, nodes1 = 512, nodes2 = 1)
model_img.compile()
model_img.fit(train_dataset, epochs = 2)
model_img.load_weights("logs/price/ensemble/final/log_price_atleast4/checkpoint")

Epoch 1/2


2022-02-21 21:28:09.584644: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - 6s 154ms/step - loss: 0.0000e+00
Epoch 2/2
22/22 [==============================] - 4s 154ms/step - loss: 0.0000e+00


2022-02-21 21:28:18.626581: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open logs/price/ensemble/final/log_price_atleast4/checkpoint: Failed precondition: logs/price/ensemble/final/log_price_atleast4/checkpoint; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


## Train Compound with reloaded weights (frozen)

In [ ]:
# Load data
X_train, X_test, X_val, y_train, y_test, y_val = load_data_fold(fold=4, drop_id = False)

In [ ]:
# merge data for tabnet and images to get consistent ids
data_df_train = pd.merge(final_df, X_train, on="id", how="inner")
data_df_train.dropna(inplace = True)
print("TRAIN SHAPE: ", data_df_train.shape)

data_df_test = pd.merge(final_df, X_test, on="id", how="inner")
data_df_test.dropna(inplace = True)
print("TEST SHAPE: ", data_df_test.shape)

data_df_val = pd.merge(final_df, X_val, on="id", how="inner")
data_df_val.dropna(inplace = True)
print("VAL SHAPE: ", data_df_val.shape)

# extract consistent columns
X_train_tab = data_df_train.filter(col_names)
X_test_tab = data_df_test.filter(col_names)
X_val_tab = data_df_val.filter(col_names)

data_df_train.drop(col_names, axis = 1, inplace = True)
data_df_test.drop(col_names, axis = 1, inplace = True)
data_df_val.drop(col_names, axis = 1, inplace = True)

# extract consistent price
y_train = data_df_train.pop("log_price")
y_test = data_df_test.pop("log_price")
y_val = data_df_val.pop("log_price")

# drop id
X_train = data_df_train.drop("id", axis = 1)
X_test = data_df_test.drop("id", axis = 1)
X_val = data_df_val.drop("id", axis = 1)


TRAIN SHAPE:  (2710, 77)
TEST SHAPE:  (853, 77)
VAL SHAPE:  (706, 77)


In [ ]:
train_size = int(X_train.shape[0] * 0.9)
batch_size = int(X_train.shape[0] * 0.1)

# X_train, X_test, y_train, y_test = train_test_split(features, prices, random_state = 123, test_size = 0.2)
# X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.2, random_state=1) # 0.25 x 0.8 = 0.2

data_train = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_train["bath"])),
                                                 "bed": np.squeeze(np.stack(X_train["bed"])),
                                                 "dining": np.squeeze(np.stack(X_train["dining"])),
                                                 "hall": np.squeeze(np.stack(X_train["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_train["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_train["living"])),
                                                 "features": X_train_tab,
                                                 "price": y_train})
data_train = data_train.cache()
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(y_train))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(batch_size)

data_test = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_test["bath"])),
                                                 "bed": np.squeeze(np.stack(X_test["bed"])),
                                                 "dining": np.squeeze(np.stack(X_test["dining"])),
                                                 "hall": np.squeeze(np.stack(X_test["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_test["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_test["living"])),
                                                 "features": X_test_tab,
                                                 "price": y_test})
data_test = data_test.cache()
test_dataset = data_test.take(len(y_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(batch_size)

data_val = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_val["bath"])),
                                                 "bed": np.squeeze(np.stack(X_val["bed"])),
                                                 "dining": np.squeeze(np.stack(X_val["dining"])),
                                                 "hall": np.squeeze(np.stack(X_val["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_val["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_val["living"])),
                                                 "features": X_val_tab,
                                                 "price": y_val})
data_val = data_val.cache()
val_dataset = data_val.take(len(y_val))
val_dataset = val_dataset.map(transform)
val_dataset = val_dataset.batch(batch_size)

In [196]:
tf.random.set_seed(2)

lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.95, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss='mse' , metrics=[R_squared,"mae"])

model.fit(train_dataset, validation_data = val_dataset, epochs=1, verbose=1)

# set layer weights to pretrained weights
model.layers[1:-4] = model_img.layers
model.layers[0] = model_tn.layers[0]

# freeze layer weights
for layer in model.layers[:-4]:
    layer.trainable = False
    
model.fit(train_dataset, validation_data = val_dataset, epochs=300, verbose=1)


Epoch 1/300


2022-02-21 21:30:31.422937: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - ETA: 0s - loss: 9.4345 - R_squared: -22.3258 - mae: 3.0027

2022-02-21 21:30:36.576661: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 8s 570ms/step - loss: 9.4345 - R_squared: -22.3258 - mae: 3.0027 - val_loss: 8.0091 - val_R_squared: -17.9431 - val_mae: 2.7875
Epoch 2/300
10/10 [==============================] - 6s 585ms/step - loss: 8.3593 - R_squared: -19.5954 - mae: 2.8145 - val_loss: 7.2311 - val_R_squared: -16.1126 - val_mae: 2.6440
Epoch 3/300
10/10 [==============================] - 5s 551ms/step - loss: 7.6460 - R_squared: -17.7314 - mae: 2.6780 - val_loss: 6.7404 - val_R_squared: -14.9769 - val_mae: 2.5475
Epoch 4/300
10/10 [==============================] - 5s 465ms/step - loss: 7.0294 - R_squared: -16.3749 - mae: 2.5571 - val_loss: 6.1967 - val_R_squared: -13.6994 - val_mae: 2.4357
Epoch 5/300
10/10 [==============================] - 5s 469ms/step - loss: 6.3974 - R_squared: -14.7609 - mae: 2.4320 - val_loss: 5.6842 - val_R_squared: -12.4846 - val_mae: 2.3261
Epoch 6/300
10/10 [==============================] - 5s 443ms/step - loss: 5.8126 - R_squared: -13.2360 - m

KeyboardInterrupt: 

### Unfreeze weights

In [197]:
for layer in model.layers[:-4]:
    layer.trainable = True
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.0001, decay_steps=100, decay_rate=0.95, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss='mse' , metrics=[R_squared,"mae"])

model.fit(train_dataset, validation_data = val_dataset, epochs=300, verbose=1)

Epoch 1/300


2022-02-21 21:38:47.148936: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - ETA: 0s - loss: 0.5776 - R_squared: -0.4180 - mae: 0.5802

2022-02-21 21:38:54.982926: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 18s 671ms/step - loss: 0.5776 - R_squared: -0.4180 - mae: 0.5802 - val_loss: 0.2736 - val_R_squared: 0.4033 - val_mae: 0.3993
Epoch 2/300
10/10 [==============================] - 6s 548ms/step - loss: 0.5732 - R_squared: -0.4072 - mae: 0.5771 - val_loss: 0.2592 - val_R_squared: 0.4383 - val_mae: 0.3870
Epoch 3/300
10/10 [==============================] - 7s 701ms/step - loss: 0.5932 - R_squared: -0.4619 - mae: 0.5861 - val_loss: 0.2511 - val_R_squared: 0.4582 - val_mae: 0.3798
Epoch 4/300
10/10 [==============================] - 7s 667ms/step - loss: 0.5630 - R_squared: -0.3790 - mae: 0.5712 - val_loss: 0.2456 - val_R_squared: 0.4718 - val_mae: 0.3747
Epoch 5/300
10/10 [==============================] - 7s 603ms/step - loss: 0.5536 - R_squared: -0.3521 - mae: 0.5685 - val_loss: 0.2427 - val_R_squared: 0.4790 - val_mae: 0.3721
Epoch 6/300
10/10 [==============================] - 7s 686ms/step - loss: 0.5486 - R_squared: -0.3422 - mae: 0.5786 - va